In [66]:
import pandas as pd
import numpy as np

from datetime import datetime

In [67]:
df1 = pd.read_csv('submit_2018-05-04_07.01.39_rerank.csv.gz')
df2 = pd.read_csv('dir_descriptor_l2_QE=2_merge_sorted_mix.csv')

In [68]:
df2['images_backup'] = df1['images'].values

In [69]:
deleted = pd.read_csv('deleted_or_offline_index_image_keys.txt', header=None)
deleted = set(deleted[0].values)

In [70]:
def process(args):
    images, images_backup = args['images'], args['images_backup']
    if not isinstance(images, str):
        return images_backup
    if not isinstance(images_backup, str):
        return images
    images = images.split(' ')
    images_backup = images_backup.split(' ')
    k = 0
    for i in range(len(images)):
        if images[i] in deleted:
            while True:
                if images_backup[k] not in deleted and images_backup[k] not in images:
                    break
                k += 1
            images[i] = images_backup[k]
    return ' '.join(images)

df2['images2'] = df2[['images', 'images_backup']].apply(process, axis=1)

In [71]:
df2.drop(['images', 'images_backup'], 1, inplace=True)
df2.rename(columns={'images2': 'images'}, inplace=True)

submit_filename = 'submit_{}_reprocessed.csv.gz'.format(datetime.now().strftime('%Y-%m-%d_%H.%M.%S'))
df2.to_csv(submit_filename, index=False, compression='gzip')